In [2]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.prompts import ChatPromptTemplate

from dotenv import load_dotenv

load_dotenv()



True

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [106]:
from langchain.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv


load_dotenv()

llm = ChatOpenAI(
    model="gpt-4o",temperature=0.9,max_tokens=500
)

#@tool
def SqlQueryWritter(user_input : str)->str:
    """
    A tool that generates SQL queries based on user input.
    """
   
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system","you are an helpfull assistent to write the SQL Queries"),
        
            ("user","{query}")
        ]
    )

    chain = prompt | llm 

    response = chain.invoke({"query":user_input})

    return response.content



@tool
def SqlQuerySummarizer(user_input : str)->str:
    """
    A tool that generates SQL queries based on user input.
    """
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system","you are an helpfull assistent , Summarize the SQL Queries"),
        
            ("user","{query}")
        ]
    )

    chain = prompt | llm 

    response = chain.invoke({"query":user_input})

    return response.content


In [81]:
tools=[SqlQueryWritter,SqlQuerySummarizer]



In [126]:
# Define the tools
tools = [SqlQueryWritter]

# Create the prompt for the agent
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "you are a helpful assistant to write queries or summarize queries"),
        ("user", "{query}"),
        ("placeholder", "{agent_scratchpad}"),
        

    ]
)

In [130]:
#from langchain.agents.initialize import initialize_agent
#from langchain.agents import create_react_agent

from langchain.agents import load_tools                                         
from langchain.agents import initialize_agent
# Create the agent using the tools
#agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    
    
)


AttributeError: 'function' object has no attribute 'is_single_input'

In [116]:
from langchain.agents import AgentExecutor
